# Tracking Failure Origins

To identify the origin of a failure, one proceeds along _dependencies_: Which earlier variables could possibly have influenced the current erroneous state? And how did their values come to be? In this chapter, we show how to identify these dependencies, and how to automatically compute them.

In [ ]:
from bookutils import YouTubeVideo
# YouTubeVideo("w4u5gCgPlmg")

**Prerequisites**

* You should have read the [Introduction to Debugging](Intro_Debugging).
* To understand how to compute dependencies automatically, advanced knowledge of Python is required.

In [ ]:
import bookutils

## Synopsis

<!-- Automatically generated. Do not edit. -->



_For those only interested in using the code in this chapter (without wanting to know how it works), give an example.  This will be copied to the beginning of the chapter (before the first section) as text with rendered input and output._

You can use `int_fuzzer()` as:

```python
print(int_fuzzer())
```
```python
=> 76.5

```


## Dependencies

We start with a simple program. The `middle()` function is supposed to return the "middle" number of three values `x`, `y`, and `z` – that is, the one number that neither is the minimum nor the maximum.

In [ ]:
def middle(x, y, z):
    if y < z:
        if x < y:
            return y
        elif x < z:
            return y
    else:
        if x > y:
            return y
        elif x > z:
            return x
    return z

In most cases, `middle()` runs just fine:

In [ ]:
m = middle(1, 2, 3)
m

In others, however, it returns the wrong value:

In [ ]:
m = middle(2, 1, 3)
m

This is a typical debugging situation: You see a value that is erroneous; and you want to find out where it came from.

To determine the origins of a value, you need to know its _dependencies_. A variable $x$ _depends_ on the value of a variable $y$ if a change in $y$ could affect the value of $x$.

We distinguish two kinds of dependencies:

* **Data dependency**: $x$ obtains its value from a computation involving the value of $y$.
* **Control dependency**: $x$ obtains its value because of a computation involving the value of $y$.

### Excursion: Visualizing Dependencies

To illustrate our examples, we introduce a `Dependencies` class that captures dependencies between variables at specific locations.

If you are reading this in a notebook, feel free to skip this excursion and directly go to "End of Excursion", below.

#### A Class for Dependencies

`Dependencies` holds two dependency graphs. `data` holds data dependencies, `control` holds control dependencies.

In [ ]:
class Dependencies(object):
    def __init__(self, data=None, control=None):
        """
Create a dependency graph from data and control. Both are dictionaries
holding _nodes_ as keys and lists of nodes as values.
Each node comes as a tuple (variable_name, location)
where `variable_name` is a string 
and `location` is a pair (code_name, lineno)
denoting a unique location in the code.
        """
        if data is None:
            data = {}
        if control is None:
            control = {}

        self.data = data
        self.control = control
        self.validate()

Each of the two is organized as a dictionary holding _nodes_ as keys and lists of nodes as values. Each node comes as a tuple

```python
(variable_name, location)
 ```
 
where `variable_name` is a string and `location` is a pair


```python
(code_name, lineno)
 ```
 
denoting a unique location in the code.

In [ ]:
class Dependencies(Dependencies):
    def validate(self):
        """Check dependency structure."""
        assert isinstance(self.data, dict)
        assert isinstance(self.control, dict)
        
        for node in (self.data.keys()) | set(self.control.keys()):
            var_name, location = node
            assert isinstance(var_name, str)
            func_name, lineno = location
            assert isinstance(func_name, str)
            assert isinstance(lineno, int)

The `source()` method returns the source code for a given node.

In [ ]:
import inspect

In [ ]:
class Dependencies(Dependencies):
    def source(self, node):
        """Return the source code for a given node."""
        (name, location) = node
        code_name, lineno = location
        fun = globals()[code_name]
        source_lines, first_lineno = inspect.getsourcelines(fun)

        try:
            line = source_lines[lineno - first_lineno].strip()
        except IndexError:
            line = ''

        return line

In [ ]:
test_deps = Dependencies()
test_deps.source(('z', ('middle', 1)))

#### Drawing Dependencies

Both data and control form a graph between nodes, and cam be visualized as such. We use the `graphviz` package for creating such visualizations.

In [ ]:
from graphviz import Digraph, nohtml

`make_graph()` sets the basic graph attributes.

In [ ]:
import html

In [ ]:
class Dependencies(Dependencies):
    NODE_COLOR = 'peachpuff'
    FONT_NAME = 'Fira Mono'

    def make_graph(self, name="dependencies", comment="Dependencies"):
        return Digraph(name=name, comment=comment, 
            graph_attr={
            },
            node_attr={
                'style': 'filled',
                'shape': 'box',
                'fillcolor': self.NODE_COLOR,
                'fontname': self.FONT_NAME
            },
            edge_attr={
                'fontname': self.FONT_NAME
            })

`graph()` returns a graph visualization.

In [ ]:
class Dependencies(Dependencies):
    def graph(self):
        """Draw dependencies."""
        self.validate()

        g = self.make_graph()
        self.draw_dependencies(g)
        self.add_hierarchy(g)
        return g

The main part of graph drawing takes place in two methods, `draw_dependencies()` and `add_hierarchy()`.

`draw_dependencies()` processes through the graph, adding nodes and edges from the dependencies.

In [ ]:
class Dependencies(Dependencies):
    def all_vars(self):
        all_vars = set()
        for var in self.data:
            all_vars.add(var)

            for source in self.data[var]:
                all_vars.add(source)

            for source in self.control[var]:
                all_vars.add(source)

        return all_vars

In [ ]:
class Dependencies(Dependencies):
    def draw_dependencies(self, g):
        for var in self.all_vars():
            g.node(self.id(var),
                   label=self.label(var),
                   tooltip=self.tooltip(var))

            if var in self.data:
                for source in self.data[var]:
                    g.edge(self.id(source), self.id(var))

            if var in self.control:
                for source in self.control[var]:
                    g.edge(self.id(source), self.id(var),
                           style='dashed', color='grey')

`draw_dependencies()` makes use of a few helper functions.

In [ ]:
class Dependencies(Dependencies):
    def id(self, var):
        """Return a unique ID for VAR."""
        id = ""
        # Avoid non-identifier characters
        for c in repr(var):
            if c.isalnum() or c == '_':
                id += c
            if c == ':' or c == ',':
                id += '_'
        return id

    def label(self, var):
        """Render node VAR using HTML style."""
        (name, location) = var
        source = self.source(var)

        title = html.escape(name)
        if name.startswith('<'):
            title = f'<I>{title}</I>'

        return (f'<'
                f'<B>{title}</B>'
                f'<FONT POINT-SIZE="9.0"><BR/><BR/>'
                f'{html.escape(source)}'
                f'</FONT>'
                f'>')

    def tooltip(self, var):
        """Return a tooltip for node VAR."""
        (name, location) = var
        code_name, lineno = location
        return f"{code_name}:{lineno}"

In the second part of graph drawing, `add_hierarchy()` adds invisible edges to ensure that nodes with lower line numbers are drawn above nodes with higher line numbers.

In [ ]:
class Dependencies(Dependencies):
    def add_hierarchy(self, g):
        """Add invisible edges for a proper hierarchy."""
        code_names = self.all_codes()
        for code_name in code_names:
            last_var = None
            last_lineno = 0
            for (lineno, var) in code_names[code_name]:
                if last_var is not None and lineno > last_lineno:
                    g.edge(self.id(last_var),
                           self.id(var),
                           style='invis')

                last_var = var
                last_lineno = lineno

        return g

In [ ]:
class Dependencies(Dependencies):
    def all_codes(self):
        code_names = {}
        for var in self.all_vars():
            (name, location) = var
            code_name, lineno = location
            if code_name not in code_names:
                code_names[code_name] = []
            code_names[code_name].append((lineno, var))

        for code_name in code_names:
            code_names[code_name].sort()

        return code_names

Here comes the graph in all its glory:

In [ ]:
def middle_deps():
    return Dependencies({('z', ('middle', 1)): set(), ('y', ('middle', 1)): set(), ('x', ('middle', 1)): set(), ('<test>', ('middle', 2)): {('y', ('middle', 1)), ('z', ('middle', 1))}, ('<test>', ('middle', 3)): {('y', ('middle', 1)), ('x', ('middle', 1))}, ('<test>', ('middle', 5)): {('z', ('middle', 1)), ('x', ('middle', 1))}, ('<middle() return value>', ('middle', 6)): {('y', ('middle', 1))}}, {('z', ('middle', 1)): set(), ('y', ('middle', 1)): set(), ('x', ('middle', 1)): set(), ('<test>', ('middle', 2)): set(), ('<test>', ('middle', 3)): {('<test>', ('middle', 2))}, ('<test>', ('middle', 5)): {('<test>', ('middle', 3))}, ('<middle() return value>', ('middle', 6)): {('<test>', ('middle', 5))}})

In [ ]:
middle_deps().graph()

#### Slices

The method `backward_slice(*items, mode='cd')` returns a subset of dependencies, following dependencies backward from the given `items`. The extra parameter `mode` controls which dependencies are to be included:

* **`d`** = data dependencies
* **`c`** = control dependencies

In [ ]:
class Dependencies(Dependencies):
    def expand_items(self, items):
        all_items = []
        for item in items:
            if isinstance(item, str):
                for var in self.all_vars():
                    (name, location) = var
                    if name == item:
                        all_items.append(var)
            else:
                all_items.append(item)

        return all_items

    def backward_slice(self, *items, mode="cd", depth=-1):
        """Create a backward slice from nodes ITEMS."""
        data = {}
        control = {}
        queue = self.expand_items(items)
        seen = set()

        while len(queue) > 0 and depth != 0:
            var = queue[0]
            queue = queue[1:]
            seen.add(var)

            if 'd' in mode:
                # Follow data dependencies
                data[var] = self.data[var]
                for next_var in data[var]:
                    if next_var not in seen:
                        queue.append(next_var)
            else:
                data[var] = set()

            if 'c' in mode:
                # Follow control dependencies
                control[var] = self.control[var]
                for next_var in control[var]:
                    if next_var not in seen:
                        queue.append(next_var)
            else:
                control[var] = set()
                
            depth = depth - 1

        return Dependencies(data, control)

### End of Excursion

### Data Dependencies

Here is an example of a data dependency in our `middle()` program. The value `y` returned by `middle()` comes from the value `y` as originally passed as argument. We use arrows $x \leftarrow y$ to indicate that a variable $x$ depends on an earlier variable $y$:

In [ ]:
# ignore
middle_deps().backward_slice('<middle() return value>', mode='d').graph()

Here, we can see that the value `y` in the return statement is data dependent on the value of `y` as passed to `middle()`. An alternate interpretation of this graph is a *data flow*: The value of `y` in the upper node _flows_ into the value of `y` in the lower node.

Since we consider the values of variables at specific locations in the program, such data dependencies can also be interpreted as dependencies between _statements_ – the above `return` statement thus is data dependent on the initialization of `y` in the upper node.

### Control Dependencies

Here is an example of a control dependency. The execution of the above `return` statement is controlled by the earlier test `x < z`. We use grey dashed lines to indicate control dependencies:

In [ ]:
# ignore
middle_deps().backward_slice('<middle() return value>', mode='c', depth=1).graph()

This test in turn is controlled by earlier tests, so the full chain of control dependencies looks like this:

In [ ]:
# ignore
middle_deps().backward_slice('<middle() return value>', mode='c').graph()

### Dependency Graphs

As the above `<test>` values (and their statements) are in turn also dependent on earlier data, namely the `x`, `y`, and `z` values as originally passed. We can draw all data and control dependencies in a single graph, called a _program dependency graph_:

In [ ]:
# ignore
middle_deps().graph()

This graph now gives us an idea on how to proceed to track the origins of the `middle()` return value at the bottom. Its value can come from any of the dependences – namely the initialization of `y` at the function call, or from the `<test>` that controls it. This test in turn depends on `x` and `z` and their associated statements, which we now can check one after the other.

Note that all these dependencies in the graph are _dynamic_ dependencies – that is, they refer to statements actually evaluated in the run at hand, as well as the decisions made in that very run. There also are _static_ dependency graphs coming from static analysis of the code; but for debugging, _dynamic_ dependencies specific to the failing run are more useful.

### Showing Dependencies with Code

While a graph gives us a representation about which possible data and control flows to track, integrating dependencies with actual program code results in a compact representation that is easy to reason about.

#### Excursion: Listing Dependencies

To show dependencies as text, we introduce a method `format_var()` that shows a single node (a variable) as text. By default, a node is referenced as

```python
NAME (FUNCTION:LINENO)
```

However, within a given function, it makes no sense to re-state the function name again and again, so we have a shorthand

```python
NAME (LINENO)
```

to state a dependency to variable `NAME` in line `LINENO`.

In [ ]:
class Dependencies(Dependencies):
    def format_var(self, var, current_location=None):
        """Return string for VAR in CURRENT_LOCATION."""
        name, location = var
        location_name, lineno = location
        if location_name != current_location:
            return f"{name} ({location_name}:{lineno})"
        else:
            return f"{name} ({lineno})"

`format_var()` is used extensively in this string representation of dependencies, listing all nodes and their data (`<=`) and control (`<-`) dependencies.

In [ ]:
class Dependencies(Dependencies):
    def __str__(self):
        self.validate()

        out = ""
        for code_name in self.all_codes():
            if out != "":
                out += "\n"
            out += f"{code_name}():\n"

            all_vars = list(set(self.data.keys()) | set(self.control.keys()))
            all_vars.sort(key=lambda var: var[1][1])

            for var in all_vars:
                (name, location) = var
                var_code, var_lineno = location
                if var_code != code_name:
                    continue

                all_deps = ""
                for (source, arrow) in [(self.data, "<="), (self.control, "<-")]:
                    deps = ""
                    for data_dep in source[var]:
                        if deps == "":
                            deps = f" {arrow} "
                        else:
                            deps += ", "
                        deps += self.format_var(data_dep, code_name)

                    if deps != "":
                        if all_deps != "":
                            all_deps += ";"
                        all_deps += deps

                if all_deps == "":
                    continue

                out += ("    " + 
                        self.format_var(var, code_name) +
                        all_deps + "\n")

        return out

    def __repr__(self):
        # Useful for saving and restoring values
        return f"Dependencies({self.data}, {self.control})"

Here is a compact string representation of dependencies. We see how the (last) `middle() return value` has a data dependency to `y` in Line 1, and to the `<test>` in Line 5.

In [ ]:
print(middle_deps())

An even more useful representation comes when integrating these dependencies as comments into the code. The method `code(item)` lists the given (function) item, including its dependencies.

In [ ]:
from bookutils import print_content

In [ ]:
class Dependencies(Dependencies):
    def code(self, item, mode='cd'):
        """List ITEM on standard output, including dependencies as comments."""
        all_vars = self.all_vars()
        slice_locations = set(location for name, location in all_vars)

        source_lines, first_lineno = inspect.getsourcelines(item)

        n = first_lineno
        for line in source_lines:
            line_location = (item.__name__, n)
            if line_location in slice_locations:
                prefix = "* "
            else:
                prefix = "  "

            print(f"{prefix}{n:4} ", end="")

            comment = ""
            for (mode_control, source, arrow) in [
                ('d', self.data, '<='),
                ('c', self.control, '<-')
            ]:
                if mode_control not in mode:
                    continue

                deps = ""
                for var in source:
                    name, location = var
                    if location == line_location:
                        for dep_var in source[var]:
                            if deps == "":
                                deps = arrow + " "
                            else:
                                deps += ", "
                            deps += self.format_var(dep_var, item.__name__)

                if deps != "":
                    if comment != "":
                        comment += "; "
                    comment += deps

            if comment != "":
                line = line.rstrip() + "  # " + comment

            print_content(line.rstrip(), '.py')
            print()
            n += 1

#### End of Excursion

The following listing shows such an integration. For each executed line (`*`), we see its data (`<=`) and control (`<-`) dependencies, listing the associated variables and line numbers. The comment

```python
# <= y (1); <- <test> (5)
```

for Line 6, for instance, states that the return value is data dependent on the value of `y` in Line 1, and control dependent on the test in Line 5.

Again, one can easily follow these dependencies back to track where a value came from (data dependencies) and why a statement was executed (control dependency).

In [ ]:
# ignore
middle_deps().code(middle)

One important aspect of dependencies is that they not only point to specific sources and causes of failures – but that they also _rule out_ parts of program and state as failures. 

* In the above code, Lines 8 and later have no influence on the output, simply because they were not executed.
* Furthermore, we see that we can start our investigation with Line 6, because that is the last one executed.
* The data dependencies tell us that no statement has interfered with the value of `y` between the function call and its return.
* Hence, the error must be in the conditions and the final `return` statement.

With this in mind, recall that our original invocation was `middle(2, 1, 3)`. Why and how is the above code wrong?

In [ ]:
from bookutils import quiz

In [ ]:
quiz("Which of the following <samp>middle()</samp> code lines should be fixed?",
    [
        "Line 2: <samp>if y < z:</samp>",
        "Line 3: <samp>if x < y:</samp>",
        "Line 5: <samp>elif x < z:</samp>",
        "Line 6: <samp>return z</samp>",
    ], (1 ** 0 + 1 ** 1) ** (1 ** 2 + 1 ** 3))

Indeed, from the controlling conditions, we see that `y < z`, `x >= y`, and `x < z` all hold. Hence, `y <= x < z` holds, and it is `x`, not `y`, that should be returned.

## Tracking Origins

The question of "Where does this value come from?" is fundamental for debugging. When programmers read code during debugging, they scan it for potential data and control dependencies. This can be a tedious experience, notably if the data and control flow is spread across multiple separate locations, possibly even in different modules.

In this chapter, we thus investigate means to _determine such dependencies_ automatically – by _collecting_ them during program execution. This means that for every variable, we need to keep track of its _origins_ – where it obtained its value, and which tests controlled its assignments.

### Wrapping Values

There are multiple ways to track such origins. One way is to _wrap_ each value in a class that stores both a value and the origin of the value. If a variable `x` is initialized to zero in Line 3, for instance, we could store it as
```
x = (value=0, origin=<Line 3>)
```
and if it is copied in, say, Line 5 to another variable `y`, we could store this as
```
y = (value=0, origin=<Line 3, Line 5>)
```
Such a scheme would allow us to track origins and dependencies right within the variable.

In a language like Python, it is actually possibly to subclass from basic types. Here's how we create a `MyInt` subclass of `int`:

In [ ]:
class MyInt(int):
    def __new__(cls, value, *args, **kwargs):
        return super(cls, cls).__new__(cls, value)

    def __repr__(self):
        return f"{int(self)}"

In [ ]:
x = MyInt(5)

We can access `x` just like any integer:

In [ ]:
x, x + 1

However, we can also add extra attributes to it:

In [ ]:
x.origin = "Line 5"

In [ ]:
x.origin

Such a "wrapping" scheme has the advantage of leaving program code untouched – simply pass "wrapped" objects instead of the original values. However, it also has a number of drawbacks.

* First, we must make sure that the "wrapper" objects are still compatible with the original values – notably by converting them back whenever needed. (What happens if an internal Python function expects an `int` and gets a `MyInt` instead?)
* Second, we have to make sure that origins do not get lost during computations – which involves overloading operators such as `+`, `-`, `*`, and so on. (Right now, `MyInt(1) + 1` gives us an `int` object, not a `MyInt`.)
* Third, we have to do this for _all_ data types of a language, which is pretty tedious.
* Fourth and last, however, we want to track whenever a value is assigned to another variable. Python has no support for this, and thus our dependencies will necessarily be incomplete.

### Tracking Data Accesses

An alternate way of tracking origins is to _instrument_ the source code such that all data read and write operations are tracked. In essence, for every occurrence of a variable `x` being read or written, we replace it with

```python
_data['x']
```

and let the `_data` object take care of maintaining the value of `x`.

Hence, an assignment such as 

```python
a = b + c
```

would get rewritten to

```python
_data['a'] = _data['b'] + _data['c']
```

and with every access to `_data`, we would track 

1. the current _location_ in the code, and 
2. whether the respective variable was read or written.

For the above statement, we could deduce that `b` and `c` were read, and `a` was written – which makes `a` data dependent on `b` and `c`.

The advantage of such instrumentation is that it works with _arbitrary objects_ (in Python, that is) – we do not case whether `a`, `b`, and `c` would be integers, floats, strings, lists. or any other type for which `+` would be defined. The disadvantage, however, is that it takes a bit of effort to exactly separate reads and writes into individual groups, and that a number of language features have to be handled separately.

### A Data Store

To implement `_data` accesses as shown above, we introduce the `DataStore` class. As its name suggests, it keeps track of variables being read and written, and provides methods to determine the code location where this tool place.

The method `caller_frame()` walks up the current call stack until it finds a method or function that is not part of `DataStore`. `caller_location()` returns its location.

In [ ]:
class DataStore(dict):
    def __init__(self, *args, log=False):
        """Initialize. If LOG is set, turn on logging."""
        super().__init__(*args)
        self.log = log

    def caller_frame(self):
        """Return the frame of the caller."""
        frame = inspect.currentframe()
        while ('self' in frame.f_locals and 
               isinstance(frame.f_locals['self'], self.__class__)):
               frame = frame.f_back
        return frame

    def caller_location(self):
        """Return the location (code name, lineno) of the caller."""
        frame = self.caller_frame()
        return frame.f_code.co_name, frame.f_lineno

`__setitem()__` is invoked when a variable is set, as in

```python
_data['pi'] = 3.1415
```

By default, we simply log the access.

In [ ]:
class DataStore(DataStore):
    def __setitem__(self, name, value):
        """Set NAME to VALUE."""
        if self.log:
            code_name, lineno = self.caller_location()
            print(f"{code_name}:{lineno}: setting {name}")

        return super().__setitem__(name, value)

`__getitem()__` is invoked when a variable is retrieved, as in

```python
print(_data['pi'])
```
By default, we simply log the access.

In [ ]:
class DataStore(DataStore):
    def __getitem__(self, name):
        """Return NAME.
        If NAME is not stored, return a local variable NAME.
        If there is no local variable NAME, 
        return the global variable NAME."""

        if self.log:
            code_name, lineno = self.caller_location()
            print(f"{code_name}:{lineno}: getting {name}")

        if name in self:
            return super().__getitem__(name)

        frame = self.caller_frame()
        if name in frame.f_locals:
            return frame.f_locals[name]

        if name in globals():
            return globals()[name]
        
        raise NameError(f"name {repr(name)} is not defined")

In [ ]:
class DataStore(DataStore):
    def __repr__(self):
        return super().__repr__()

Here's an example of a logging `DataStore`:

In [ ]:
_test_data = DataStore(log=True)
_test_data['x'] = 1

In [ ]:
_test_data['x']

If a variable is defined outside of the data store, we can still access it:

In [ ]:
y = 3
_test_data['y']

Accessing unknown names gives us a `NameError`:

In [ ]:
from ExpectError import ExpectError

In [ ]:
with ExpectError():
    _test_data['z']

### Instrumenting Source Code

How do we transform source code such that read and write accesses to variables would be automatically rewritten?  To this end, we inspect the internal representation of source code, namely the _abstract syntax trees_ (ASTs). An AST represents the code as a tree, with specific node types for each syntactical element.

In [ ]:
import ast
import astor

Here is the tree representation for our `middle()` function. It starts with a `FunctionDef` node at the top (with the name `"middle"` and the three arguments `x`, `y`, `z` as children), followed by a subtree for each of the `If` statements, each of which contains a branch for when their condition evaluates to`True` and a branch for when their condition evaluates to `False`.

In [ ]:
from bookutils import rich_output

In [ ]:
if rich_output():
    from showast import show_ast
else:
    def show_ast(tree):
        ast.dump(tree)

In [ ]:
middle_tree = ast.parse(inspect.getsource(middle))
show_ast(middle_tree)

At the very bottom of the tree, you can see a number of `Name` nodes, referring individual variables. These are the ones we want to transform.

### Tracking Variable Access

Our goal is to _traverse_ the tree, identify all `Name` nodes, and convert them to respective `_data` accesses.
To this end, we manipulate the AST through the Python modules `ast` and `astor`. The [official Python `ast` reference](http://docs.python.org/3/library/ast) is complete, but a bit brief; the documentation ["Green Tree Snakes - the missing Python AST docs"](https://greentreesnakes.readthedocs.io/en/latest/) provides an excellent introduction.

The Python `ast` class provides a class `NodeTransformer` that allows such transformations. Subclassing from it, we provide a method `visit_Name()` that will be invoked for all `Name` nodes – and replace it by a new subtree from `make_data_access()`:

In [ ]:
from ast import NodeTransformer

In [ ]:
class TrackAccessTransformer(NodeTransformer):
    def visit_Name(self, node):
        if node.id in dir(__builtins__):
            # Do not change built-in names
            return node

        return make_data_access(node.id, node.ctx)

Our function `make_data_access(id)` returns a new subtree equivalent to the Python code `_data[id]`.

In [ ]:
from ast import Module, Subscript, Name, Load, Store, \
    Assign, Attribute, With, withitem, Return, Index, Str, Call, Expr

In [ ]:
DATA_STORE = '_data'

In [ ]:
def make_data_access(id, ctx):
    return Subscript(
        value=Name(id=DATA_STORE, ctx=Load()),
        slice=Index(value=Str(s=id)),
        ctx=ctx
    )

This is the tree that `make_data_access()` produces:

In [ ]:
show_ast(Module(body=[make_data_access("x", Load())]))

How do we know that this is a correct subtree? We can carefully read the [official Python `ast` reference](http://docs.python.org/3/library/ast) and then proceed by trial and error (and apply [delta debugging](DeltaDebugger.ipynb) to determine error causes). Or – pro tip! – we can simply take a piece of Python code, parse it and use `ast.dump()` to print out how to construct the resulting AST:

In [ ]:
print(ast.dump(ast.parse("_data['x']")))

If you compare the above output with the code of `make_data_access()`, above, you will find out where the source of `make_data_access()` comes from.

Let us put `TrackAccessTransFormer` to action. Its `visit()` method calls `visit_Name()`, which then in turn transforms the `Name` nodes as we want it. This happens in place.

In [ ]:
TrackAccessTransformer().visit(middle_tree);

To see the effect of our transformations, we introduce a method `dump_tree()` which outputs the tree – and also compiles it to check for any inconsistencies.

In [ ]:
def dump_tree(tree):
    print(astor.to_source(tree))
    ast.fix_missing_locations(tree)  # Must run this before compiling
    _ = compile(tree, '<string>', 'exec')

We see that our transformer has properly replaced all variable references:

In [ ]:
dump_tree(middle_tree)

Let us now execute this code together with the `DataStore()` class we previously introduced. The class `TreeTester()` takes a (transformed) tree and a function. Using it as

```python
with TreeTester(tree, func):
    func(...)
```

first executes the code in _tree_ (possibly instrumenting `func`) and then the `with` body. At the end, `func` is restored to its previous (non-instrumented) version.

In [ ]:
class TreeTester(object):
    def __init__(self, tree, func):
        self.code = compile(tree, '<string>', 'exec')
        self.func = func

    def __enter__(self):
        globals()[DATA_STORE] = DataStore(log=True)
        exec(self.code, globals())
    
    def __exit__(self, exc_type, exc_value, traceback):
        globals()[self.func.__name__] = self.func
        del globals()[DATA_STORE]

Here is our `middle()` function:

In [ ]:
print_content(inspect.getsource(middle), '.py', start_line_number=1)

And here is our instrumented `middle_tree` executed with a `DataStore` object. We see how the `middle()` tests access one argument after another.

In [ ]:
with TreeTester(middle_tree, middle):
    middle(2, 1, 3)

After `TreeTester` is done, `middle` is reverted to its non-instrumented version:

In [ ]:
middle(2, 1, 3)

For a complete picture of what happens during executions, we implement a number of additional code transformers.

For each function argument `x`, we insert a call `_data.param('x', x)` to initialize `x`. This is useful for __tracking arguments across function calls.__

### Excursion: Tracking Arguments

For the remaining transformers, we follow the same steps as for `TrackAccessTransformer`, except that our `visit_...()` methods focus on different nodes, and return different subtrees. Again, all these subtrees were obtained from `ast.dump()`:

In [ ]:
print(ast.dump(ast.parse("_data.param('x', x)")))

In [ ]:
class TrackParamsTransformer(NodeTransformer):
    def visit_FunctionDef(self, node):
        self.generic_visit(node)

        named_args = []
        for child in ast.iter_child_nodes(node.args):
            if isinstance(child, ast.arg):
                named_args.append(child.arg)

        create_stmts = []
        for arg in named_args:
            create_stmt = Expr(
                value=Call(
                    func=Attribute(value=Name(id=DATA_STORE, ctx=Load()),
                                   attr='param', ctx=Load()),
                    args=[Str(s=arg), Name(id=arg, ctx=Load())],
                    keywords=[]
                )
            )
            create_stmts.append(create_stmt)
        create_stmts.reverse()

        node.body = create_stmts + node.body
        return node

This is the effect of `TrackParamsTransformer()`. You see how the first three parameters are all initialized.

In [ ]:
TrackParamsTransformer().visit(middle_tree)
dump_tree(middle_tree)

By default, the `DataStore` `param()` method simply calls `__setitem__()` to set variables.

In [ ]:
class DataStore(DataStore):
    def param(self, name, value):
        if self.log:
            code_name, lineno = self.caller_location()
            print(f"{code_name}:{lineno}: initializing {name}")

        return self.__setitem__(name, value)

In [ ]:
with TreeTester(middle_tree, middle):
    middle(2, 1, 3)

### End of Excursion

For each return statement `return x`, we insert a call `_data['<return value>'] = x` before the statement. This allows to __track return values__.

### Excursion: Tracking Return Values

In [ ]:
class TrackReturnTransformer(NodeTransformer):
    def __init__(self):
        self.function_name = None
        super().__init__()

    def visit_FunctionDef(self, node):
        self.function_name = node.name  # Save current function name
        self.generic_visit(node)
        return node

    def return_value(self):
        if self.function_name is None:
            return "<return value>"
        else:
            return f"<{self.function_name}() return value>"

    def visit_Return(self, node):
        assign_node = Assign(
                targets=[make_data_access(self.return_value(), Store())],
                value=node.value
            )
        return_node = Return(
                value=make_data_access(self.return_value(), Load())
            )
        ast.copy_location(assign_node, node)
        ast.copy_location(return_node, node)

        return [
            assign_node,
            return_node
        ]

This is the effect of `TrackReturnTransformer`. We see that all return values are saved, and thus all locations of the corresponding return statements are tracked.

In [ ]:
TrackReturnTransformer().visit(middle_tree)
dump_tree(middle_tree)

In [ ]:
with TreeTester(middle_tree, middle):
    middle(2, 1, 3)

### End of Excursion

To track __control dependencies__, for every block controlled by an `if`, `while`, or `for`:

1. We wrap their tests in a `_data.test()` wrapper. This allows us to assign pseudo-variables like `<test>` which hold the conditions.
2. We wrap their controlled blocks in a `with` statement. This allows us to track the variables read right before the `with` (= the controlling variables), and to restore the current controlling variables when the block is left.

A statement

```python
if cond:
    body
```

thus becomes

```python
if _data.test(cond):
    with _data:
        body
```

### Excursion: Tracking Control

Again, all these subtrees are obtained via `ast.dump()`.

In [ ]:
class TrackControlTransformer(NodeTransformer):
    def make_with(self, block):
        if len(block) == 0:
            return []

        return [With(
            items=[
                withitem(
                    context_expr=Name(id=DATA_STORE, ctx=Load()),
                    optional_vars=None)
            ],
            body=block
        )]

    def make_test(self, test):
        return Call(func=Attribute(value=Name(id=DATA_STORE, ctx=Load()),
                                   attr='test',
                                   ctx=Load()),
                     args=[test],
                     keywords=[])

    def visit_If(self, node):
        self.generic_visit(node)
        node.test = self.make_test(node.test)
        node.body = self.make_with(node.body)
        node.orelse = self.make_with(node.orelse)
        return node

In [ ]:
class TrackControlTransformer(TrackControlTransformer):
    def visit_While(self, node):
        self.generic_visit(node)
        node.test = self.make_test(node.test)
        node.body = self.make_with(node.body)
        node.orelse = self.make_with(node.orelse)
        return node

In [ ]:
class TrackControlTransformer(TrackControlTransformer):
    def visit_For(self, node):
        self.generic_visit(node)
        node.body = self.make_with(node.body)
        node.orelse = self.make_with(node.orelse)
        return node

Here is the effect of `TrackControlTransformer`:

In [ ]:
TrackControlTransformer().visit(middle_tree)
dump_tree(middle_tree)

We extend `DataStore` to also log these events:

In [ ]:
class DataStore(DataStore):
    def test(self, cond):
        if self.log:
            code_name, lineno = self.caller_location()
            print(f"{code_name}:{lineno}: testing condition")

        return cond

In [ ]:
class DataStore(DataStore):
    def __enter__(self):
        if self.log:
            code_name, lineno = self.caller_location()
            print(f"{code_name}:{lineno}: entering block")

    def __exit__(self, exc_type, exc_value, traceback):
        if self.log:
            code_name, lineno = self.caller_location()
            print(f"{code_name}:{lineno}: exiting block")

In [ ]:
with TreeTester(middle_tree, middle):
    middle(2, 1, 3)

### End of Excursion

Finally, we want to be able to __track calls__ across multiple functions. To this end, we wrap each call

```python
func(arg1, arg2, ...)
```

into

```python
_data.ret(_data.call(func)(_data.arg(arg1), _data.arg(arg2), ...))
```

each of which simply pass through their given argument, but which allow to track the beginning of calls (`call()`), the computation of arguments (`arg()`), and the return of the call (`ret()`), respectively.

### Tracking Calls and Arguments

In [ ]:
class TrackCallTransformer(NodeTransformer):
    def make_call(self, node, fun):
        return Call(func=Attribute(value=Name(id=DATA_STORE,
                                              ctx=Load()),
                                   attr=fun,
                                   ctx=Load()),
                     args=[node],
                     keywords=[])

    def visit_Call(self, node):
        self.generic_visit(node)

        new_args = []
        for arg in node.args:
            new_args.append(self.make_call(arg, 'arg'))
        node.args = new_args

        for kw in node.keywords:
            kw.value = self.make_call(kw.value, 'arg')

        node.func = self.make_call(node.func, 'call')
        return self.make_call(node, 'ret')

Our example function `middle()` does not contain any calls, but here is a function that invokes `middle()` twice:

In [ ]:
def test_call():
    x = middle(1, 2, middle(1, 2, 3))

In [ ]:
call_tree = ast.parse(inspect.getsource(test_call))
dump_tree(call_tree)

If we invoke `TrackCallTransformer` on this testing function, we get the following transformed code:

In [ ]:
TrackCallTransformer().visit(call_tree);

In [ ]:
dump_tree(call_tree)

As before, our default `arg()`, `ret()`, and `call()` methods simply log the event and pass through the given value.

In [ ]:
class DataStore(DataStore):
    def arg(self, value):
        if self.log:
            code_name, lineno = self.caller_location()
            print(f"{code_name}:{lineno}: pushing arg")

        return value

In [ ]:
class DataStore(DataStore):
    def ret(self, value):
        if self.log:
            code_name, lineno = self.caller_location()
            print(f"{code_name}:{lineno}: returned from call")

        return value

In [ ]:
class DataStore(DataStore):
    def call(self, func):
        if self.log:
            code_name, lineno = self.caller_location()
            print(f"{code_name}:{lineno}: calling {func}")

        return func

In [ ]:
with TreeTester(call_tree, test_call):
    test_call()

### End of Excursion

What do we obtain after we have applied all these transformers on `middle()`? We see that the code now contains quite a load of instrumentation:

In [ ]:
dump_tree(middle_tree)

And when we execute this code, we see that we can track quite a number of events:

In [ ]:
with TreeTester(middle_tree, middle):
    middle(2, 1, 3)

Our next step will now be not only to _log_ these events, but to actually construct _dependencies_ from them.

## Building Dependencies

In [ ]:
import itertools

In [ ]:
class DataTracker(DataStore):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.origins = {}
        self.data_dependencies = {}
        self.control_dependencies = {}

        self.data = [[]]  # Data stack
        self.control = [[]]  # Control stack
        self.last_read = []
        self.last_checked_location = None
        self._ignore_location_change = False

        self.args = []  # Argument stack

### Data Dependencies

In [ ]:
class DataTracker(DataTracker):
    def clear_read(self):
        if self.log:
            direct_caller = inspect.currentframe().f_back.f_code.co_name
            code_name, lineno = self.caller_location()
            print(f"{code_name}:{lineno}: "
                  f"clearing read variables {self.last_read} "
                  f"(from {direct_caller})")

        self.last_read = []

    def check_location(self):
        location = self.caller_location()
        if self.last_checked_location != location:
            if self._ignore_location_change:
                self._ignore_location_change = False
            else:
                self.clear_read()

        self.last_checked_location = location

    def ignore_next_location_change(self):
        self._ignore_location_change = True

    def ignore_location_change(self):
        self.last_checked_location = self.caller_location()

    def __getitem__(self, name):
        self.check_location()
        self.last_read.append(name)
        return super().__getitem__(name)

In [ ]:
class DataTracker(DataTracker):
    def __setitem__(self, name, value):
        def add_dependencies(dependencies, vars_read, tp):
            for var_read in vars_read:
                if var_read in self.origins:
                    origin = self.origins[var_read]
                    dependencies.add((var_read, origin))
                    if self.log:
                        origin_name, origin_lineno = origin
                        code_name, lineno = self.caller_location()
                        print(f"{code_name}:{lineno}: "
                              f"new {tp} dependency: "
                              f"{name} <= {var_read}@"
                              f"{origin_name}:{origin_lineno}")

        self.check_location()
        ret = super().__setitem__(name, value)
        location = self.caller_location()

        if (name, location) not in self.data_dependencies:
            self.data_dependencies[(name, location)] = set()
        if (name, location) not in self.control_dependencies:
            self.control_dependencies[(name, location)] = set()

        add_dependencies(self.data_dependencies[(name, location)],
                         self.last_read, tp="data")
        add_dependencies(self.control_dependencies[(name, location)],
                         itertools.chain.from_iterable(self.control),
                         tp="control")

        self.origins[name] = location

        # Reset read info for next line
        self.clear_read()

        return ret

### Control Dependencies

In [ ]:
class DataTracker(DataTracker):
    TEST = '<test>'

    def test(self, value):
        self.__setitem__(self.TEST, value)
        self.__getitem__(self.TEST)
        return super().test(value)

    def __enter__(self):
        self.control.append(self.last_read)
        self.clear_read()
        super().__enter__()

    def __exit__(self, exc_type, exc_value, traceback):
        self.clear_read()
        self.last_read = self.control.pop()
        self.ignore_next_location_change()
        super().__exit__(exc_type, exc_value, traceback)

    def dependencies(self):
        return Dependencies(self.data_dependencies,
                            self.control_dependencies)

### Calls and Returns

In [ ]:
class DataTracker(DataTracker):
    def call(self, fun):
        # Save context
        if self.log:
            code_name, lineno = self.caller_location()
            print(f"{code_name}:{lineno}: "
                  f"saving read variables {self.last_read}")

        self.data.append(self.last_read)
        self.clear_read()
        self.ignore_next_location_change()

        return super().call(fun)

    def ret(self, value):
        super().ret(value)

        # Restore old context and add return value
        ret_name = None
        for var in self.last_read:
            if var.startswith("<"):
                ret_name = var

        self.last_read = self.data.pop()
        self.last_read.append(ret_name)
        self.ignore_location_change()

        if self.log:
            code_name, lineno = self.caller_location()
            print(f"{code_name}:{lineno}: "
                  f"restored read variables {self.last_read}")

        return value

### Function Arguments

In [ ]:
class DataTracker(DataTracker):
    def arg(self, value):
        if self.log:
            code_name, lineno = self.caller_location()
            print(f"{code_name}:{lineno}: "
                  f"saving arg reads {self.last_read}")

        self.args.append(self.last_read)
        self.clear_read()
        return super().arg(value)

    def param(self, name, value):
        self.clear_read()
        if self.args:
            self.last_read = self.args.pop()
            self.ignore_location_change()

        if self.log:
            code_name, lineno = self.caller_location()
            print(f"{code_name}:{lineno}: "
                  f"restored param {self.last_read}")

        return super().param(name, value)

### Diagnostics

In [ ]:
import re
import sys

In [ ]:
class Dependencies(Dependencies):
    def validate(self):
        super().validate()

        for var in self.all_vars():
            source = self.source(var)
            for dep_var in self.data[var] | self.control[var]:
                dep_name, dep_location = dep_var

                if dep_name == DataTracker.TEST:
                    continue

                if dep_name.endswith('return value>'):
                    if source.find('(') < 0:
                        print(f"Warning: {self.format_var(var)} "
                              f"depends on {self.format_var(dep_var)}, "
                              f"but {repr(source)} does not "
                              f"seem to have a call",
                              file=sys.stderr
                             )
                    continue

                if source.startswith('def'):
                    continue   # function call

                rx = re.compile(r'\b' + dep_name + r'\b')
                if rx.search(source) is None:
                    print(f"Warning: {self.format_var(var)} "
                          f"depends on {self.format_var(dep_var)}, "
                          f"but {repr(dep_name)} does not occur "
                          f"in {repr(source)}",
                          file=sys.stderr
                         )

## Slicing Code

In [ ]:
class Instrumenter(object):
    def __init__(self, *items_to_instrument, log=False):
        self.log = log
        self.items_to_instrument = items_to_instrument

    def __enter__(self):
        """Instrument sources"""
        for item in self.items_to_instrument:
            self.instrument(item)
        return self

    def __exit__(self, exc_type, exc_value, traceback):
        """Restore sources"""
        self.restore()

    def restore(self):
        for item in self.items_to_instrument:
            globals()[item.__name__] = item

    def instrument(self, item):
        if self.log:
            print("Instrumenting", item)

In [ ]:
with Instrumenter(middle) as ins:
    pass

In [ ]:
class Slicer(Instrumenter):
    def __init__(self, *items_to_instrument, 
                 data_tracker=None,
                 log=False):
        super().__init__(*items_to_instrument, log=log)
        if len(items_to_instrument) == 0:
            raise ValueError("Need one or more items to instrument")

        if data_tracker is None:
            data_tracker = DataTracker(log=(log > 1))
        self.data_tracker = data_tracker
        self.data_store = None

    def instrument(self, item):
        source_lines, lineno = inspect.getsourcelines(item)
        tree = ast.parse("".join(source_lines))
        ast.increment_lineno(tree, lineno - 1)

        TrackAccessTransformer().visit(tree)
        TrackCallTransformer().visit(tree)
        TrackControlTransformer().visit(tree)
        TrackReturnTransformer().visit(tree)
        TrackParamsTransformer().visit(tree)

        ast.fix_missing_locations(tree)
        # print_ast_ids(tree)

        if self.log:
            print(f"Instrumenting {item}:")

            if self.log > 1:
                n = lineno
                for line in source_lines:
                    print(f"{n:4} {line.rstrip()}")
                    n += 1
                print()

            print(astor.to_source(tree))

        code = compile(tree, '<string>', 'exec')
        exec(code, globals())
        globals()[DATA_STORE] = self.data_tracker

    def restore(self):
        if DATA_STORE in globals():
            self.data_store = globals()[DATA_STORE]
            del globals()[DATA_STORE]
        super().restore()

    def dependencies(self):
        if self.data_store is None:
            return Dependencies({}, {})
        return self.data_store.dependencies()

    def code(self, *args, **kwargs):
        first = True
        for item in self.items_to_instrument:
            if not first:
                print()
            self.dependencies().code(item, *args, **kwargs)
            first = False

    def graph(self, *args, **kwargs):
        return self.dependencies().graph(*args, **kwargs)

In [ ]:
with Slicer(middle) as slicer:
    m = middle(2, 1, 3)
m

In [ ]:
print(slicer.dependencies())

In [ ]:
slicer.dependencies()

In [ ]:
slicer.code()

In [ ]:
middle(2, 1, 3)

In [ ]:
with Slicer(middle) as middle_slicer:
    y = middle(2, 1, 3)

In [ ]:
middle_slicer.graph()

In [ ]:
print(middle_slicer.dependencies())

## More Examples

In [ ]:
from Assertions import square_root

In [ ]:
import math

In [ ]:
with Slicer(square_root, log=True) as root_slicer:
    y = square_root(2.0)

In [ ]:
root_slicer.graph()

FIXME: How does `<test>` control itself?

In [ ]:
root_slicer.code()

In [ ]:
root_slicer.dependencies()

In [ ]:
from Intro_Debugging import remove_html_markup

In [ ]:
with Slicer(remove_html_markup) as rhm_slicer:
    s = remove_html_markup("<foo>bar</foo>")

In [ ]:
rhm_slicer.graph()

In [ ]:
rhm_slicer.code()

In [ ]:
rhm_slicer.dependencies().backward_slice('tag', mode='c').graph()

In [ ]:
def add_to(n, m):
    n += m
    return n

In [ ]:
def mul_with(x, y):
    x *= y
    return x

In [ ]:
def test_math():
    return mul_with(1, add_to(2, 2))

In [ ]:
with Slicer(add_to, mul_with, test_math) as math_slicer:
    test_math()

In [ ]:
math_slicer.graph()

In [ ]:
math_slicer.code()

In [ ]:
print(math_slicer.dependencies())

## Things that do not Work

### Data Structures

In [ ]:
def test_array():
    x = [1, 2, 3]
    y = 4
    x[2] = y
    return x[2]

In [ ]:
with Slicer(test_array, log=True) as array_slicer:
    test_array()

In [ ]:
array_slicer.graph()

### Attributes

In [ ]:
class X(object):
    pass

In [ ]:
def test_attributes(y):
    x = X()
    x.attr = y
    return x.attr

In [ ]:
with Slicer(test_attributes, log=True) as attr_slicer:
    test_attributes(10)

In [ ]:
attr_slicer.graph()

TODO: Auto-slice things – in 

```python
with Slicer():
   foo()
```
identify all calls in the code and add them to the instrumentation.

1. Get source code - won't work; will not contain body
2. Use a decorator? (works only for functions...)
3. Replace on the fly – won't work (or will it?), as we cannot change code once it executes

In [ ]:
from ExpectError import ExpectError

In [ ]:
with ExpectError():
    with Slicer() as slicer:
        y = square_root(9)

TODO: Create a probabilistic dependence graph for statistical debugging :-)

## Synopsis

_For those only interested in using the code in this chapter (without wanting to know how it works), give an example.  This will be copied to the beginning of the chapter (before the first section) as text with rendered input and output._

## Lessons Learned

* _Lesson one_
* _Lesson two_
* _Lesson three_

## Next Steps

_Link to subsequent chapters (notebooks) here, as in:_

* [use _mutations_ on existing inputs to get more valid inputs](MutationFuzzer.ipynb)
* [use _grammars_ (i.e., a specification of the input format) to get even more valid inputs](Grammars.ipynb)
* [reduce _failing inputs_ for efficient debugging](Reducer.ipynb)


## Background

_Cite relevant works in the literature and put them into context, as in:_

The idea of ensuring that each expansion in the grammar is used at least once goes back to Burkhardt \cite{Burkhardt1967}, to be later rediscovered by Paul Purdom \cite{Purdom1972}.

@INPROCEEDINGS{6899220,
  author={Z. {Chen} and L. {Chen} and Y. {Zhou} and Z. {Xu} and W. C. {Chu} and B. {Xu}},
  booktitle={2014 IEEE 38th Annual Computer Software and Applications Conference}, 
  title={Dynamic Slicing of Python Programs}, 
  year={2014},
  volume={},
  number={},
  pages={219-228},
  doi={10.1109/COMPSAC.2014.30}
}



## Exercises

_Close the chapter with a few exercises such that people have things to do.  To make the solutions hidden (to be revealed by the user), have them start with_

```
**Solution.**
```

_Your solution can then extend up to the next title (i.e., any markdown cell starting with `#`)._

_Running `make metadata` will automatically add metadata to the cells such that the cells will be hidden by default, and can be uncovered by the user.  The button will be introduced above the solution._

### Exercise 1: _Title_

_Text of the exercise_

In [ ]:
# Some code that is part of the exercise
pass

_Some more text for the exercise_

**Solution.** _Some text for the solution_

In [ ]:
# Some code for the solution
2 + 2

_Some more text for the solution_

### Exercise 2: _Title_

_Text of the exercise_

**Solution.** _Solution for the exercise_